# Analysis of the DEIS programme in Ireland
### Part 1: Loading and Preparing Data for Analysis

In [2]:
#Loading required libraries
library(readxl)
library(dplyr)
library(fixest)
library(purrr)
library(broom)
library(modelsummary)
library(tidyr)
library(ggplot2)
library(meta)
library(stringr)
library(writexl)

In [ ]:
#Accessing dataset
#This is a saved datafile which combines all DoE post-primary school information with the SPA information and Clusters information prepared in previous 2 codefiles

tryCatch({
  df<- read_excel("C:\\") # Load the dataset with your combined schools, SPA and clusters data
  message("Loaded main data")
}, error = function(e) {
  stop("Unable to load", e$message)
})    

In [ ]:
#Checking the column names for and ensure they match were required throughout analyses
colnames(df) 

### Part 2: Pre-DEIS Implementation:
#### Plotting means of DEIS vs non-DEIS for SPA, KNN and non-grouped schools
##### These plots visualise the mean annual percent change in graduation class size of all DEIS schools against all non-DEIS schools.
##### There are three plots, the first looks at all schools separated only by DEIS status from 2006 onwards, the second looks at DEIS vs. non-DEIS schools when the means of each SPA cluster are calculated then the overall mean of all clusters, and the final plot repeats this but clusters are formed by KNN grouping.

In [ ]:
#Plot 1: Visualising the data of Mean Annual Percent Change in Graduation Class Size of all DEIS schools vs all non-DEIS schools

dataFiltered <- dataFiltered %>%
  mutate(DEIS_factor = factor(ifelse(DEIS == 1, "DEIS", "Non-DEIS"),
                             levels = c("Non-DEIS", "DEIS"))) #DEIS and non-DEIS grouping

#Plot for all schools without clustering
overall_plot_data <- dataFiltered %>%
  group_by(Year, DEIS_factor) %>%
  summarise(Mean_pcChange = mean(pcChange, na.rm = TRUE), .groups = "drop")

overall_plot <- ggplot(overall_plot_data, aes(x = Year, y = Mean_pcChange, color = DEIS_factor)) +
  geom_line(linewidth = 1) +
  geom_point(size = 2) +
  scale_color_manual(values = c("Non-DEIS" = "purple", "DEIS" = "green")) +
  geom_vline(xintercept = c(2006, 2017), linetype = "dotted", color = "black", linewidth = 1) +
  labs(
    title = "Mean % Change in Graduation Class Size Over Time (All Schools)",
    x = "Year",
    y = "Mean % Change in Graduation Class Size",
    color = "DEIS Status"
  ) +
  theme_minimal() +
  theme(legend.position = "bottom")

ggsave("All_Schools_DEIS_vs_NonDEIS.png", plot = overall_plot, width = 12, height = 8, dpi = 300)

In [ ]:
#Plot 2: Mean Annual Percent Change in Graduation Class size of DEIS vs. non-DEIS schools when grouped by SPA
# Ensure DEIS is a factor with levels "Non-DEIS" and "DEIS"
dataFiltered$DEIS_label <- factor(dataFiltered$DEIS, levels = c(0, 1), labels = c("Non-DEIS", "DEIS"))

# Calculate mean percentage change for each KNN cluster and DEIS status
spa_means <- dataFiltered %>%
  group_by(SPA_ID, DEIS_label) %>%
  summarise(Mean_pcChange = mean(pcChange, na.rm = TRUE), .groups = "drop") %>%
  arrange(SPA_ID, DEIS_label)

# View the calculated means
print(spa_means)

spaPlot<-ggplot(spa_means, aes(x = SPA_ID, y = Mean_pcChange, color = DEIS_label, group = DEIS_label)) +
  geom_line(size = 1) +
  geom_point(size = 2) +
  scale_color_manual(values = c("Non-DEIS" = "purple", "DEIS" = "green")) +
  labs(
    title = "Mean % Change in Graduation Class Size by KNN Cluster and DEIS Status",
    x = "KNN Cluster (clusterED5)",
    y = "Mean % Change in Graduation Class Size",
    color = "School Type"
  ) +
  theme_minimal() +
  theme(legend.position = "bottom")

ggsave("spa.png", plot = spaPlot, width = 12, height = 8, dpi = 300)

In [ ]:
#Plot 3: Mean Annual Percent Change in Graduation class size of DEIS vs. non-DEIS schools when grouped by KNN
dataFiltered$DEIS_label <- factor(dataFiltered$DEIS, levels = c(0, 1), labels = c("Non-DEIS", "DEIS"))

# Calculate mean percentage change for each KNN cluster and DEIS status
knn_means <- dataFiltered %>%
  group_by(clusterED5, DEIS_label) %>%
  summarise(Mean_pcChange = mean(pcChange, na.rm = TRUE), .groups = "drop") %>%
  arrange(clusterED5, DEIS_label)

# View the calculated means
print(knn_means)
#KNN plot
knnPlot<-ggplot(knn_means, aes(x = clusterED5, y = Mean_pcChange, color = DEIS_label, group = DEIS_label)) +
  geom_line(size = 1) +
  geom_point(size = 2) +
  scale_color_manual(values = c("Non-DEIS" = "purple", "DEIS" = "green")) +
  labs(
    title = "Mean % Change in Graduation Class Size by KNN Cluster and DEIS Status",
    x = "KNN Cluster (clusterED5)",
    y = "Mean % Change in Graduation Class Size",
    color = "School Type"
  ) +
  theme_minimal() +
  theme(legend.position = "bottom")

ggsave("knn.png", plot = knnPlot, width = 12, height = 8, dpi = 300)

### Checking the parallel trends assumption; first holding 2006 as the post-treatment year, then 2012.

In [ ]:
#Code to plot DEIS vs non-DEIS schools pre-2006

parallel6<- ggplot(df %>% filter(Year < 2006), aes(x = Year, y = pcChange, color = as.factor(DEIS))) +
  stat_summary(fun = mean, geom = "line") +
  labs(title = "Pre-Treatment Trends by DEIS Status")

#Table of pre-treatment trends
pre_trend_table1 <- df %>%
  filter(Year < 2006) %>%             
  group_by(Year, DEIS) %>%            
  summarise(
    mean_pcChange= mean(pcChange, na.rm = TRUE),
    sd_pcChange = sd(pcChange, na.rm = TRUE),
    n = n(),
    se_pcChange = sd_pcChange / sqrt(n) 
  ) %>%
  ungroup()
print(pre_trend_table1)

df_pre1<- df %>% filter(Year < 2006)
df_pre1<- df_pre1 %>% mutate(year_c = Year - 2000)
pre_trend_test1<- lm(pcChange ~ year_c * DEIS + Gender + indexDif, data = df_pre1)
summary(pre_trend_test1)
ggsave("pre_treatment_trends6.png", plot = parallel6, width = 8, height = 6, dpi = 300)

In [ ]:
parallel12<- ggplot(df %>% filter(Year < 2012), aes(x = Year, y = pcChange, color = as.factor(DEIS))) +
  stat_summary(fun = mean, geom = "line") +
  labs(title = "Pre-Treatment Trends by DEIS Status")

#Table of pre-treatment trends
pre_trend_table1 <- df %>%
  filter(Year < 2006) %>%             
  group_by(Year, DEIS) %>%            
  summarise(
    mean_pcChange= mean(pcChange, na.rm = TRUE),
    sd_pcChange = sd(pcChange, na.rm = TRUE),
    n = n(),
    se_pcChange = sd_pcChange / sqrt(n) 
  ) %>%
  ungroup()
print(pre_trend_table1)

df_pre1<- df %>% filter(Year < 2012)
df_pre1<- df_pre1 %>% mutate(year_c = Year - 2000)
pre_trend_test1<- lm(pcChange ~ year_c * DEIS + Gender + indexDif, data = df_pre1)
summary(pre_trend_test1)
ggsave("pre_treatment_trends12.png", plot = parallel12, width = 8, height = 6, dpi = 300)

In [ ]:
# Running an event study around the 2012 cohort to check for any jumps/spikes/changes before 2012
df <- df %>%
  mutate(rel_year = Year - 2012,
         treatment = DEIS)
dfCohort12 <- df %>%
  filter(rel_year >= -5, rel_year <= 5) #Checks 5 years pre and 5 years post 2012 cohort graduation

dfCohort12 <- dfCohort12 %>%
  mutate(rel_year_factor = factor(rel_year)) %>%
  mutate(rel_year_factor = relevel(rel_year_factor, ref = "0"))
eventCohort12<- lm(pcChange ~ rel_year_factor * treatment + Gender + indexDif, data = dfCohort12) #Regression
eventCoefCohort12<- tidy(eventCohort12) %>%
  filter(str_detect(term, "rel_year_factor") & str_detect(term, "treatment")) 
eventCoefCohort12<- eventCoefCohort12%>%
  mutate(rel_year = as.numeric(str_extract(term, "-?\\d+")))  

#Show estimated treatment effect for each year
cohort12table <- eventCoefCohort12 %>%
  select(rel_year, estimate, std.error, statistic, p.value) %>%
  mutate(
    conf.low = estimate - 1.96 * std.error,
    conf.high = estimate + 1.96 * std.error
  ) %>%
  arrange(rel_year)
print(cohort12table)
#Plot results
eventStudy2012Plot <- ggplot(eventCoefCohort12, aes(x = rel_year, y = estimate)) +
  geom_point() +
  geom_errorbar(aes(ymin = estimate - 1.96 * std.error,
                    ymax = estimate + 1.96 * std.error), width = 0.2) +
  geom_vline(xintercept = 0, linetype = "dashed", color = "light green") +
  geom_hline(yintercept = 0, linetype = "dotted") +
  labs(title = "Event Study: DEIS First Fully-Treated Cohort (2012)",
       x = "Years from 2012 Cohort",
       y = "Treatment Effect (DEIS vs non-DEIS)",
       caption = "Zero line = no effect; green dashed = first cohort year") +
  theme_minimal()

ggsave("eventStudy2012Plot.png", plot = eventStudy2012Plot, width = 8, height = 6, dpi = 300)

## Part 3: 
## Hypothesis 1 - The DEIS programme improved educational retention
### First, overall look at the DiD and inverse variance results of both post-2006 and post-2012, as well as all 8 permutations of school clustering:

In [ ]:
#First, for 2006
#Establishing the different analysis permutations
postTreatment2006<- c(2006)#Creating threshold variable
clusters<-c("SPA_ID", "clusterED5", "pobalRankCluster0.05", 
          "pobalRankCluster0.1", "pobalRankCluster0.2", "pobalRankCluster0.5",
          "pobalRankCluster1.0") ##Creating variable to loop over the cluster groups - change to your own variable names
df$Gender <- factor(df$Gender, levels = c(0, 1, 2), 
                    labels = c("Mixed", "All-Girls", "All-Boys")) #Renames gender variable

didPermutations <- function(df, postTreatment, clusterVar) {
  df <- df %>%
    mutate(
      post = ifelse(Year >= postTreatment, 1, 0),
      treatment = DEIS
    )
  
  df %>%
    group_by(across(all_of(clusterVar))) %>%
    group_modify(~{
      dfGrouped <- .x
      formula <- if (n_distinct(dfGrouped$Gender) < 2) {
        pcChange ~ treatment * post + indexDif
      } else {
        pcChange ~ treatment * post + Gender + indexDif
      }
      model <- lm(formula, data = dfGrouped)
      tidy(model)
    }) %>%
    ungroup()
}

resultsFull <- cross_df(list(
  postTreatment = postTreatment2006,
  clusterVar = clusters
)) %>%
  mutate(results = map2(postTreatment, clusterVar, ~ didPermutations(df, .x, .y))) %>%
  unnest(results) #This loops over the full set of treatment year-cluster permutations

ivw <- resultsFull %>%
  filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
  group_by(postTreatment, clusterVar) %>%
  mutate(weight = 1 / (std.error^2)) %>%
  summarise(
    ivwEst = sum(weight * estimate) / sum(weight),
    ivwSE = sqrt(1 / sum(weight)),
    confLower = ivwEst - qnorm(0.975) * ivwSE,
    confUpper = ivwEst + qnorm(0.975) * ivwSE,
    .groups = "drop"
  )
print(ivw)
#Saving these results:
write_xlsx(ivw, "C:\\x") #Add file path and name dataset
message("Saved results as Excel file.")

In [ ]:
# Repeating for restricted event study period (2007-2017) for 2012 threshold
df_filtered <- df %>% filter(Year >= 2007, Year <= 2017) #reducing period to surround event study
df_filtered$Gender <- factor(df_filtered$Gender, 
                             levels = c(0, 1, 2), 
                             labels = c("Mixed", "All-Girls", "All-Boys"))

postTreatment2012<- c(2012) # Define new post-treatment year 

didPermutations <- function(df, postTreatment, clusterVar) {
  df <- df %>%
    mutate(
      post = ifelse(Year >= postTreatment, 1, 0),
      treatment = DEIS
    )

  df %>%
    group_by(across(all_of(clusterVar))) %>%
    group_modify(~{
      dfGrouped <- .x
      formula <- if (n_distinct(dfGrouped$Gender) < 2) {
        pcChange ~ treatment * post + indexDif
      } else {
        pcChange ~ treatment * post + Gender + indexDif
      }
      model <- lm(formula, data = dfGrouped)
      tidy(model)
    }) %>%
    ungroup()
}

resultsFull12 <- cross_df(list(
  postTreatment = postTreatment2012,
  clusterVar = clusters
)) %>%
  mutate(results = map2(postTreatment, clusterVar, 
                        ~ didPermutations(df_filtered, .x, .y))) %>%
  unnest(results) %>%
  mutate(postTreatmentIter = postTreatment, 
         clusterVarIter = clusterVar)

ivw12 <- resultsFull12 %>%
  filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
  group_by(postTreatmentIter, clusterVarIter) %>%
  mutate(weight = 1 / (std.error^2)) %>%
  summarise(
    ivwEst = sum(weight * estimate) / sum(weight),
    ivwSE = sqrt(1 / sum(weight)),
    confLower = ivwEst - qnorm(0.975) * ivwSE,
    confUpper = ivwEst + qnorm(0.975) * ivwSE,
    .groups = "drop"
  )

# Print and save results
print(ivw12)
write_xlsx(ivw12, "C:/") #Add your file path and name dataset

### In-depth analysis of each group of school clusters with post-treatment years of 2006 and 2012:
#### First for schools grouped by neighbouring Electoral Districts

In [ ]:
didFunction1 <- function(df, postTreat, cluster = "clusterED5") {
  
  df <- df %>%
    mutate(
      post = ifelse(Year >= postTreat, 1, 0),
      treatment = DEIS
    )
  
  df$Gender <- factor(df$Gender, levels = c(0, 1, 2), labels = c("Mixed", "All-Girls", "All-Boys"))

  # Run model per cluster
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      data <- .x
      # Drop Gender if only 1 level
      if (n_distinct(data$Gender) < 2) {
        model <- lm(pcChange ~ treatment * post + indexDif, data = data)
      } else {
        model <- lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
      }
      broom::tidy(model)
    }) %>%
    ungroup()

  # Meta-analysis / IVW estimate
  results1 <- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))

  ivwEst1 <- with(results1, sum(weight * estimate) / sum(weight))
  ivwSE1 <- sqrt(1 / sum(results1$weight))
  z1 <- qnorm(0.975)
  confLower1 <- ivwEst1 - z1 * ivwSE1
  confUpper1 <- ivwEst1 + z1 * ivwSE1

  cat("IVW Estimate for post year", postTreat, ":", ivwEst1, "\n")
  cat("95% CI:", confLower1, "to", confUpper1, "\n\n")

  # Plot histogram
  hist(results1$estimate, breaks = 50,
       main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  
  print(summary(results1$estimate))

  # Meta-analysis
  meta_res1 <- metagen(
    TE = estimate,
    seTE = std.error,
    data = results1,
    sm = "MD"
  )
  
  print(summary(meta_res1))
  forest(meta_res1, main = paste("Forest Plot of Treatment Effects for post =", postTreat))

  # ANOVA with Gender (only if >1 level in full df)
  if (n_distinct(df$Gender) < 2) {
    formula1 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + indexDif"))
  } else {
    formula1 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  }

  model1 <- lm(formula1, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  anova_res <- anova(model1)
  print(anova_res)

  return(list(
    ivwEst1 = ivwEst1,
    confLower1 = confLower1,
    confUpper1 = confUpper1,
    results1 = results1,
    meta_res1 = meta_res1,
    anova = anova_res
  ))
}
results12006 <- didFunction1(df, 2006)
# results12012 <- didFunction1(df_filtered, 2012) #Repeat this on restricted dataset for robustness check

In [ ]:
#Generating then saving ANOVA tables
anovahyp1KNN2006 <- tidy(results12006$anova)
#anovahyp1KNN2012 <- tidy(results12012$anova) # Robustness check
write_xlsx(
  list(
    ANOVA_2006 = anovahyp1KNN2006#,
    #ANOVA_2012 = anovahyp1KNN2012 #Robustness check
  ),
  path = "C:/" #Add your filepath and name
)

#### Then for schools grouped by Electoral Districts within 0.01 points of each other on the Pobal scale
###### *This is disregarded from analysis as clusters created are two small for any meangingful results to be drawn*

In [ ]:
didFunction2<- function(df, postTreat, cluster = "pobalRankCluster0.01") {
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did2<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did2(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results2<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst2<- with(results2, sum(weight * estimate) / sum(weight))
  ivwSE2<- sqrt(1 / sum(results2$weight))
  z2<- qnorm(0.975)
  confLower2<- ivwEst2- z2* ivwSE2
  confUpper2<- ivwEst2+ z2* ivwSE2
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst2, "\n")
  cat("95% CI:", confLower2, "to", confUpper2, "\n\n")
  
  hist(results2$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results2$estimate))
  
  meta_res2<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results2,
    sm = "MD"
  )
  
  print(summary(meta_res2))
  forest(meta_res2, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula2<- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model2<- lm(formula2, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model2))
  
  return(list(ivwEst2 = ivwEst2,
              confLower2= confLower2,
              confUpper2 = confUpper2,
              results2 = results2,
              meta_res2= meta_res2,
              anova = anova(model2)))
}

results22006<- didFunction2(df, 2006)
# results22012 <- didFunction2(df, 2012) #Robustness check

#### Then for schools grouped by Electoral Districts within 0.05 points of each other on the Pobal scale

In [ ]:
didFunction3<- function(df, postTreat, cluster = "pobalRankCluster0.05") { #Change to reflect name of grouping to nearest 0.05 Pobal 
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did3<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did3(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results3<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst3<- with(results3, sum(weight * estimate) / sum(weight))
  ivwSE3<- sqrt(1 / sum(results3$weight))
  z3<- qnorm(0.975)
  confLower3<- ivwEst3- z3* ivwSE3
  confUpper3<- ivwEst3+ z3* ivwSE3
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst3, "\n")
  cat("95% CI:", confLower3, "to", confUpper3, "\n\n")
  
  hist(results3$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results3$estimate))
  
  meta_res3<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results3,
    sm = "MD"
  )
  
  print(summary(meta_res3))
  forest(meta_res3, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula3 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model3<- lm(formula3, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model3))
  
  return(list(ivwEst3 = ivwEst3,
              confLower3= confLower3,
              confUpper3 = confUpper3,
              results3 = results3,
              meta_res3= meta_res3,
              anova = anova(model3)))
}

results32006<- didFunction3(df, 2006)
# results32012 <- didFunction3(df, 2012) #Robustness check

#### Then for schools grouped by Electoral Districts within 0.1 points of each other on the Pobal scale

In [ ]:
didFunction4<- function(df, postTreat, cluster = "pobalRankCluster0.1") { #Change to refletc column name of grouping to nearest 0.1 Pobal score from your dataset
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did4<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did4(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results4<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst4<- with(results4, sum(weight * estimate) / sum(weight))
  ivwSE4<- sqrt(1 / sum(results4$weight))
  z4<- qnorm(0.975)
  confLower4<- ivwEst4- z4* ivwSE4
  confUpper4<- ivwEst4+ z4* ivwSE4
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst4, "\n")
  cat("95% CI:", confLower4, "to", confUpper4, "\n\n")
  
  hist(results4$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results4$estimate))
  
  meta_res4<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results4,
    sm = "MD"
  )
  
  print(summary(meta_res4))
  forest(meta_res4, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula4<- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model4<- lm(formula4, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model4))
  
  return(list(ivwEst4 = ivwEst4,
              confLower4= confLower4,
              confUpper4 = confUpper4,
              results4 = results4,
              meta_res4= meta_res4,
              anova = anova(model4)))
}

results42006<- didFunction4(df, 2006)
# results42012 <- didFunction4(df, 2012) #Robustness check

#### Then for schools grouped by Electoral Districts within 0.2 points of each other on the Pobal scale

In [ ]:
didFunction5<- function(df, postTreat, cluster = "pobalRankCluster0.2") { #Change to reflect clustering to nearest 0.2 Pobal score from your dataset
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did5<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did5(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results5<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst5<- with(results5, sum(weight * estimate) / sum(weight))
  ivwSE5<- sqrt(1 / sum(results5$weight))
  z5<- qnorm(0.975)
  confLower5<- ivwEst5- z5* ivwSE5
  confUpper5<- ivwEst5+ z5* ivwSE5
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst5, "\n")
  cat("95% CI:", confLower5, "to", confUpper5, "\n\n")
  
  hist(results5$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results5$estimate))
  
  meta_res5<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results5,
    sm = "MD"
  )
  
  print(summary(meta_res5))
  forest(meta_res5, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula5 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model5<- lm(formula5, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model5))
  
  return(list(ivwEst5 = ivwEst5,
              confLower5= confLower5,
              confUpper5 = confUpper5,
              results5 = results5,
              meta_res5= meta_res5,
              anova = anova(model5)))
}

results52006<- didFunction5(df, 2006)
# results52012 <- didFunction5(df, 2012) #Robustness check

#### Then for schools grouped by Electoral Districts within 0.5 points of each other on the Pobal scale

In [ ]:
didFunction6<- function(df, postTreat, cluster = "pobalRankCluster0.5") { #Change to reflect grouping to nearest 0.5 Pobal scaore from your dataset
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did6<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did6(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results6<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst6<- with(results6, sum(weight * estimate) / sum(weight))
  ivwSE6<- sqrt(1 / sum(results6$weight))
  z6<- qnorm(0.975)
  confLower6<- ivwEst6- z6* ivwSE6
  confUpper6<- ivwEst6+ z6* ivwSE6
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst6, "\n")
  cat("95% CI:", confLower6, "to", confUpper6, "\n\n")
  
  hist(results6$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results6$estimate))
  
  meta_res6<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results6,
    sm = "MD"
  )
  
  print(summary(meta_res6))
  forest(meta_res6, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula6 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model6<- lm(formula6, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model6))
  
  return(list(ivwEst6 = ivwEst6,
              confLower6= confLower6,
              confUpper6 = confUpper6,
              results6 = results6,
              meta_res6= meta_res6,
              anova = anova(model6)))
}

results62006<- didFunction6(df, 2006)
# results62012 <- didFunction6(df, 2012) #Robustness check

#### Then for schools grouped by Electoral Districts within 1.0 points of each other on the Pobal scale

In [ ]:
didFunction7<- function(df, postTreat, cluster = "pobalRankCluster1") { #Change to match grouping to nearest 1.0 degree from your dataset
    
  df <- df %>%
    mutate(post = ifelse(Year >= postTreat, 1, 0),
           treatment = DEIS)
  
  did7<- function(data) {
    lm(pcChange ~ treatment * post + Gender + indexDif, data = data)
  }
  
  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      model <-did7(.x)
      tidy(model)
    }) %>%
    ungroup()
  
  results7<- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))
  
  ivwEst7<- with(results7, sum(weight * estimate) / sum(weight))
  ivwSE7<- sqrt(1 / sum(results7$weight))
  z7<- qnorm(0.975)
  confLower7<- ivwEst7- z7* ivwSE7
  confUpper7<- ivwEst7+ z7* ivwSE7
  
  cat("IVW Estimate for post year", postTreat, ":", ivwEst7, "\n")
  cat("95% CI:", confLower7, "to", confUpper7, "\n\n")
  
  hist(results7$estimate, breaks = 50, main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results7$estimate))
  
  meta_res7<- metagen(
    TE = estimate,
    seTE = std.error,
    data = results7,
    sm = "MD"
  )
  
  print(summary(meta_res7))
  forest(meta_res7, main = paste("Forest Plot of Treatment Effects for post =", postTreat))
  
  formula7 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  model7<- lm(formula7, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model7))
  
  return(list(ivwEst7 = ivwEst7,
              confLower7= confLower7,
              confUpper7 = confUpper7,
              results7= results7,
              meta_res7= meta_res7,
              anova = anova(model7)))
}

results72006<- didFunction7(df, 2006)
# results72012 <- didFunction7(df, 2012) #Robustness check

#### Finally, schools grouped based on the School Planning Area they are assigned to

In [ ]:
didFunction8 <- function(df, postTreat, cluster = "SPA_ID") { #Change to SPA cluster name from your dataset
  
  df <- df %>%
    mutate(
      post = ifelse(Year >= postTreat, 1, 0),
      treatment = DEIS
    )

  df$Gender <- factor(df$Gender, levels = c(0, 1, 2), labels = c("Mixed", "All-Girls", "All-Boys"))

  results <- df %>%
    group_by(across(all_of(cluster))) %>%
    group_modify(~{
      data <- .x
      # Use Gender from the group, not full df
      if (n_distinct(data$Gender) < 2) {
        formula8 <- pcChange ~ treatment * post + indexDif
      } else {
        formula8 <- pcChange ~ treatment * post + Gender + indexDif
      }
      model8 <- lm(formula8, data = data)
      broom::tidy(model8)
    }) %>%
    ungroup()

  results8 <- results %>%
    filter(term == "treatment:post", !is.na(estimate), !is.na(std.error)) %>%
    mutate(weight = 1 / (std.error^2))

  ivwEst8 <- with(results8, sum(weight * estimate) / sum(weight))
  ivwSE8 <- sqrt(1 / sum(results8$weight))
  z8 <- qnorm(0.975)
  confLower8 <- ivwEst8 - z8 * ivwSE8
  confUpper8 <- ivwEst8 + z8 * ivwSE8

  cat("IVW Estimate for post year", postTreat, ":", ivwEst8, "\n")
  cat("95% CI:", confLower8, "to", confUpper8, "\n\n")

  hist(results8$estimate, breaks = 50,
       main = paste("Distribution of Treatment Effects for post =", postTreat),
       xlab = "Treatment Effect (treatment:post)", col = "skyblue")
  print(summary(results8$estimate))

  meta_res8 <- metagen(
    TE = estimate,
    seTE = std.error,
    data = results8,
    sm = "MD"
  )

  print(summary(meta_res8))
  forest(meta_res8, main = paste("Forest Plot of Treatment Effects for post =", postTreat))

  # Conditional ANOVA
  if (n_distinct(df$Gender) < 2) {
    formula8 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + indexDif"))
  } else {
    formula8 <- as.formula(paste0("pcChange ~ treatment * post * factor(", cluster, ") + Gender + indexDif"))
  }

  model8 <- lm(formula8, data = df)
  cat("\nANOVA, post =", postTreat, ":\n")
  print(anova(model8))

  return(list(
    ivwEst8 = ivwEst8,
    confLower8 = confLower8,
    confUpper8 = confUpper8,
    results8 = results8,
    meta_res8 = meta_res8,
    anova = anova(model8)
  ))
}
results82006 <- didFunction8(df, 2006)
results82012 <- didFunction8(df, 2012)

anovahyp1SPA2006 <- tidy(results82006$anova)
# anovahyp1SPA2012 <- tidy(results82012$anova) #Robustness check
write_xlsx(
  list(
    ANOVA_2006 = anovahyp1SPA2006,
   # ANOVA_2012 = anovahyp1SPA2012 #Robustness check
  ),
  path = "C:/" #Add your filepath, name and save
)

## Part 4: 
## Hypothesis 2 - The DEIS reforms from 2017 onwards improved educational retention
### The data from 2017-2022 does not allow for a fully exposed cohort to be assessed, thus only one post-treatment grouping

In [ ]:
# First issue: Parallel Trends Assumption does not hold.
# Running an event study around the 2017 reform to check for any jumps/spikes/changes before 2017
df <- df %>%
  mutate(rel_year = Year - 2017,
         treatment = DEIS)
df_event <- df %>%
  filter(rel_year >= -5, rel_year <= 5) #Checks 5 years pre and 5 years post 2017 reform

df_event <- df_event %>%
  mutate(rel_year_factor = factor(rel_year)) %>%
  mutate(rel_year_factor = relevel(rel_year_factor, ref = "0"))
event_model <- lm(pcChange ~ rel_year_factor * treatment + Gender + indexDif, data = df_event) #Regression
event_coefs <- tidy(event_model) %>%
  filter(str_detect(term, "rel_year_factor") & str_detect(term, "treatment")) 
event_coefs <- event_coefs %>%
  mutate(rel_year = as.numeric(str_extract(term, "-?\\d+")))  # Extract year from term name

#Show estimated treatment effect for each year
Hyp2table <- event_coefs %>%
  select(rel_year, estimate, std.error, statistic, p.value) %>%
  mutate(
    conf.low = estimate - 1.96 * std.error,
    conf.high = estimate + 1.96 * std.error
  ) %>%
  arrange(rel_year)
print(Hyp2table)
#Plot results
eventStudy2017Plot <- ggplot(event_coefs, aes(x = rel_year, y = estimate)) +
  geom_point() +
  geom_errorbar(aes(ymin = estimate - 1.96 * std.error,
                    ymax = estimate + 1.96 * std.error), width = 0.2) +
  geom_vline(xintercept = 0, linetype = "dashed", color = "purple") +
  geom_hline(yintercept = 0, linetype = "dotted") +
  labs(title = "Event Study: DEIS Reform (2017)",
       x = "Years from 2017 Reform",
       y = "Treatment Effect (DEIS vs non-DEIS)",
       caption = "Zero line = no effect; purple dashed = reform year") +
  theme_minimal()

ggsave("eventStudy2017Plot.png", plot = eventStudy2017Plot, width = 8, height = 6, dpi = 300)

In [ ]:
# Narrower window and school fixed analysis for hyp 2 to reduce noise
df_subset <- df %>%
  filter(Year >= 2014 & Year <= 2020) %>%
  mutate(
    post = ifelse(Year >= 2017, 1, 0),
    treatment = DEIS,
    treat_post = treatment * post
  ) #Narrower window to reduce noise
did_fe <- feols(pcChange ~ treat_post + indexDif | `School Code` + Year, data = df_subset)
summary(did_fe)
summary(did_fe, cluster = ~`School Code`) #clustering by school to assess fixed effects
modelsummary(did_fe, output = "hyp2didResults.html") #saving the table

In [ ]:
#Repeating controlling for clusters:
cluster_vars <- c("SPA_ID", "clusterED5", "pobalRankCluster0.05", 
                  "pobalRankCluster0.1", "pobalRankCluster0.2", 
                  "pobalRankCluster0.5", "pobalRankCluster1.0") #Change these cluster names to reflect column names in your dataset

models <- list()

for (clust_var in cluster_vars) {
  fe_formula <- as.formula(
    paste("pcChange ~ treatment * post + indexDif |", clust_var, "+ Year") #Change to reflect names from your dataset
  )
  models[[clust_var]] <- feols(
    fe_formula,
    data = df_subset,
    cluster = clust_var
  )
}

In [ ]:
#Visualising the clusters:
cluster_vars <- c("SPA", "KNN ED", "HP Deprivation 0.05", 
                  "HP Deprivation 0.1", "HP Deprivation 0.2", 
                  "HP Deprivation 0.5", "HP Deprivation 1.0") #Change to reflect names from your dataset
clusterGroups<- cluster_vars
did_results <- data.frame(
  Cluster = clusterGroups,
  Estimate = c(-39.553, -38.742, -54.488, -50.871, -47.006, -42.231, -41.968),
  StdError = c(5.679, 6.11, 32.276, 31.286, 28.337, 25.287, 22.566)
)
did_results <- did_results %>%
  mutate(
    CI_Lower = Estimate - 1.96 * StdError,
    CI_Upper = Estimate + 1.96 * StdError
  )
# Calculate confidence intervals
did_results <- did_results %>%
  mutate(
    CI_Lower = Estimate - 1.96 * StdError,
    CI_Upper = Estimate + 1.96 * StdError
  )

# Plot
hyp2cluster<-ggplot(did_results, aes(x = Estimate, y = reorder(Cluster, Estimate))) +
  geom_point(color = "purple", size = 3) +
  geom_errorbarh(aes(xmin = CI_Lower, xmax = CI_Upper), height = 0.3, color = "grey40") +
  geom_vline(xintercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Estimated Treatment Effects (2017 DEIS Reform)",
    x = "Treatment Effect on % Change in Graduation Size",
    y = "Clustering Method"
  ) +
  theme_minimal()
ggsave("hyp2clusterPlot.png", plot = hyp2cluster, width = 8, height = 6, dpi = 300)